In [1]:
import numpy as np
import os
from sklearn.metrics import mean_squared_error as mse_f
from scipy import sparse
from scipy.stats import gamma
from scipy.stats import ttest_ind
import warnings
import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
paper_model_names = models = {
    "dsbmm_dpf.z-only": "DPIF-DD-Topic+Net",
    "dsbmm_dpf.z-theta-joint": "DPIF-DD-Topic+Net+Pubs",
    # 'spf.main':'MSPF',
    "unadjusted.main": "Unadjusted",
    "network_pref_only.main": "Network Only",
    "topic_only.main": "DPIF-DD-Topic",
    "no_unobs.main": "Oracle",
    "topic_only_oracle.main": "DPIF-Topic (No region)",
}


In [3]:
tmp_kv = tuple(paper_model_names.items())
for k, v in tmp_kv:
    if "dsbmm_dpf" in k:
        paper_model_names[k + "-ndc"] = v + " (No DC)"

tmp_dict = {
    k + "old_subs" + "_ewcnone" + "_rcolmain_adm1_1hot": v
    for k, v in paper_model_names.items()
    if "dsbmm_dpf" not in k
}
sub_choice_pretty = {"old_subs": " (prev. subs.) ", "upd_subs": " (subs. w. trans.) "}
reg_choice_pretty = {"adm1": "(ADM1)", "ctry": "(Ctry.)"}
for k, v in paper_model_names.items():
    if "dsbmm_dpf" in k:
        for sub_choice in ["old_subs", "upd_subs"]:
            for region in ["adm1", "ctry"]:
                tmp_dict[k + f"{sub_choice}_ewcnone_rcolmain_{region}_1hot"] = (
                    v + sub_choice_pretty[sub_choice] + reg_choice_pretty[region]
                )


In [4]:
full_paper_model_names = tmp_dict

In [54]:
def print_table(exp_results, regimes, models, exps=10, print_notfound=False):
    ncols = len(regimes.keys())
    nrows = len(models)
    results = np.zeros((nrows, ncols))
    std = np.zeros((nrows, ncols))
    alt_results = np.zeros((nrows, ncols))
    alt_std = np.zeros((nrows, ncols))

    col_idx = 0
    for regime, c in regimes.items():
        row_idx = 0
        for model in models:
            mse = np.zeros((exps,4))
            for i in range(exps):
                try:
                    beta_predicted = exp_results[c][model][i][0]
                    truth = exp_results[c][model][i][1]
                    sq_err = (beta_predicted - truth) ** 2
                    mse[i] = sq_err.mean(axis=0)
                except:
                    if print_notfound:
                        print(model, "exp", i, "not found")
            results[row_idx][col_idx] = round(mse.mean() * 1000, 2)
            std[row_idx][col_idx] = round(mse.std() * 1000, 2)
            
            alt_results[row_idx][col_idx] = round(mse[:,:-1].mean() * 1000, 2)
            alt_std[row_idx][col_idx] = round(mse[:,:-1].std() * 1000, 2)
            
            row_idx += 1
        col_idx += 1

    proper_names = [full_paper_model_names[m] for m in models]
    col_names = list(regimes.keys())
    df = pd.DataFrame(results, index=proper_names, columns=col_names, dtype=str)
    std_df = pd.DataFrame(std, index=proper_names, columns=col_names, dtype=str)
    df = df + "$\pm$" + std_df
    
    alt_df = pd.DataFrame(alt_results, index=proper_names, columns=col_names, dtype=str)
    alt_std_df = pd.DataFrame(alt_std, index=proper_names, columns=col_names, dtype=str)
    alt_df = alt_df + "$\pm$" + alt_std_df
    return df, alt_df


### Load results

In [55]:
from pathlib import Path

res_dir = Path("/scratch/fitzgeraldj/data/caus_inf_data/results")
exps = 5
# embed = "user"
sub_choices = ["old_subs","upd_subs"] 
regions = ["adm1","ctry"]  # or "ctry" for dsbmm_dpf models
base_models = [
    "unadjusted.main",
    "network_pref_only.main",
    "topic_only.main",
    "no_unobs.main",
    "topic_only_oracle.main",
    "dsbmm_dpf.z-only",
    "dsbmm_dpf.z-theta-joint",
    "dsbmm_dpf.z-theta-joint-ndc",
]
models = [m + f"{sub_choice}_ewcnone_rcolmain_{region}_1hot" for m in base_models for sub_choice in sub_choices for region in regions]

conf_types = ["homophily", "exog", "both"]
confounding_strengths = [(50, 10), (50, 50), (50, 100)]
exp_results = {}
found = set()

for i in range(1, exps + 1):
    for model in models:
        for (cov1conf, cov2conf) in confounding_strengths:
            for ct in conf_types:
                try:
                    base_file_name = (
                        "conf="
                        + str((cov1conf, cov2conf))
                        + ";conf_type="
                        + ct
                        + ".npz"
                    )
                    result_file = (
                        (res_dir / str(i)) / (model + "_model_fitted_params")
                    ) / base_file_name
                    res = np.load(result_file)
                    params = res["fitted"]
                    truth = res["true"]

                    if (ct, (cov1conf, cov2conf)) in exp_results:
                        if model in exp_results[(ct, (cov1conf, cov2conf))]:
                            exp_results[(ct, (cov1conf, cov2conf))][model].append(
                                (params, truth)
                            )
                        else:
                            exp_results[(ct, (cov1conf, cov2conf))][model] = [
                                (params, truth)
                            ]
                    else:
                        exp_results[(ct, (cov1conf, cov2conf))] = {
                            model: [(params, truth)]
                        }
                    if model not in found:
                        print(model, "found")
                        found |= set([model])
                except:
                    # print(result_file, " not found")
                    continue


dsbmm_dpf.z-theta-jointold_subs_ewcnone_rcolmain_adm1_1hot found
dsbmm_dpf.z-theta-jointold_subs_ewcnone_rcolmain_ctry_1hot found
unadjusted.mainold_subs_ewcnone_rcolmain_adm1_1hot found
no_unobs.mainold_subs_ewcnone_rcolmain_adm1_1hot found
topic_only_oracle.mainold_subs_ewcnone_rcolmain_adm1_1hot found


In [56]:
# dsbmm_dpf.z-onlyupd_subs_ewcnone_rcolmain_adm1_1hot_model_fitted_params
found

{'dsbmm_dpf.z-theta-jointold_subs_ewcnone_rcolmain_adm1_1hot',
 'dsbmm_dpf.z-theta-jointold_subs_ewcnone_rcolmain_ctry_1hot',
 'no_unobs.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'topic_only_oracle.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'unadjusted.mainold_subs_ewcnone_rcolmain_adm1_1hot'}

### Visualize results

In [ ]:
confounding_type = "exog"
models = list(exp_results[(confounding_type, confounding_strengths[1])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df1,alt_df1 = print_table(exp_results, regime1, models)


In [ ]:
confounding_type = "homophily"
models = list(exp_results[(confounding_type, confounding_strengths[0])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df2,alt_df2 = print_table(exp_results, regime1, models)


In [ ]:
confounding_type = "both"
models = list(exp_results[(confounding_type, confounding_strengths[0])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df3,alt_df3 = print_table(exp_results, regime1, models)


In [60]:
all_results = pd.concat([df1, df2, df3], axis=1, keys=["Exog.", "Homophily", "Both"])
all_results


Exog.                 \
                                                        Low           Med.   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   1.38$\pm$2.95  2.28$\pm$5.56   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  1.38$\pm$2.95  2.28$\pm$5.56   
Unadjusted                                    0.17$\pm$0.76  0.14$\pm$0.46   
Oracle                                        0.18$\pm$0.97  0.09$\pm$0.42   
DPIF-Topic (No region)                        0.04$\pm$0.19  0.04$\pm$0.21   

                                                              \
                                                        High   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   3.79$\pm$13.43   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  3.79$\pm$13.43   
Unadjusted                                     0.13$\pm$0.42   
Oracle                                         0.28$\pm$1.62   
DPIF-Topic (No region)                         0.12$\pm$0.67   

                                                       Homophily  \
                                                             Low   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)    491.2$\pm$1821.58   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  394.03$\pm$1368.99   
Unadjusted                                     110.31$\pm$682.04   
Oracle                                        170.47$\pm$1054.85   
DPIF-Topic (No region)                         162.0$\pm$1002.77   

                                                                 \
                                                           Med.   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   203.81$\pm$679.71   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  203.81$\pm$679.71   
Unadjusted                                    109.78$\pm$678.65   
Oracle                                        116.88$\pm$723.92   
DPIF-Topic (No region)                        122.91$\pm$761.32   

                                                                 \
                                                           High   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   103.36$\pm$326.75   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  103.36$\pm$326.75   
Unadjusted                                      79.5$\pm$490.85   
Oracle                                          88.36$\pm$548.8   
DPIF-Topic (No region)                         89.99$\pm$557.69   

                                                            Both  \
                                                             Low   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   490.99$\pm$1839.56   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  490.99$\pm$1839.56   
Unadjusted                                    193.31$\pm$1194.58   
Oracle                                        241.21$\pm$1493.81   
DPIF-Topic (No region)                        218.24$\pm$1350.98   

                                                                  \
                                                            Med.   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)    227.33$\pm$718.77   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)   227.33$\pm$718.77   
Unadjusted                                     130.43$\pm$806.33   
Oracle                                          139.52$\pm$865.2   
DPIF-Topic (No region)                        173.62$\pm$1075.91   

                                                                 
                                                           High  
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   122.18$\pm$391.01  
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  122.18$\pm$391.01  
Unadjusted                                     94.02$\pm$580.34  
Oracle                                        103.21$\pm$640.38  
DPIF-Topic (No region)                         99.77$\pm$617.87

In [67]:
all_results.to_latex('./results/semi-synth.tex')

In [61]:
alt_all_results = pd.concat([alt_df1, alt_df2, alt_df3], axis=1, keys=["Exog.", "Homophily", "Both"])
alt_all_results


Exog.                 \
                                                        Low           Med.   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   0.81$\pm$2.26  1.24$\pm$3.76   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  0.81$\pm$2.26  1.24$\pm$3.76   
Unadjusted                                    0.07$\pm$0.22  0.11$\pm$0.34   
Oracle                                         0.03$\pm$0.1  0.03$\pm$0.11   
DPIF-Topic (No region)                        0.01$\pm$0.03  0.01$\pm$0.04   

                                                                  Homophily  \
                                                       High             Low   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)    1.07$\pm$2.4  6.11$\pm$12.13   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)   1.07$\pm$2.4  6.56$\pm$12.94   
Unadjusted                                     0.1$\pm$0.33    1.43$\pm$5.4   
Oracle                                        0.03$\pm$0.12   2.03$\pm$10.8   
DPIF-Topic (No region)                        0.01$\pm$0.04   1.86$\pm$9.26   

                                                                            \
                                                       Med.           High   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   4.47$\pm$8.64  4.31$\pm$8.31   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  4.47$\pm$8.64  4.31$\pm$8.31   
Unadjusted                                    1.44$\pm$5.46  1.18$\pm$4.13   
Oracle                                        1.26$\pm$6.68  0.63$\pm$3.35   
DPIF-Topic (No region)                        1.31$\pm$6.08   0.9$\pm$3.69   

                                                        Both                 \
                                                         Low           Med.   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   6.23$\pm$12.34  4.82$\pm$9.19   
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  6.23$\pm$12.34  4.82$\pm$9.19   
Unadjusted                                    2.64$\pm$11.58  1.71$\pm$6.77   
Oracle                                        2.62$\pm$13.92  1.27$\pm$6.75   
DPIF-Topic (No region)                        2.49$\pm$12.63  1.75$\pm$8.41   

                                                             
                                                       High  
DPIF-DD-Topic+Net+Pubs (prev. subs.) (ADM1)   4.97$\pm$9.38  
DPIF-DD-Topic+Net+Pubs (prev. subs.) (Ctry.)  4.97$\pm$9.38  
Unadjusted                                     1.42$\pm$5.3  
Oracle                                         0.87$\pm$4.6  
DPIF-Topic (No region)                        1.08$\pm$4.68

In [66]:
alt_all_results.to_latex('./results/alt-semi-synth.tex')

In [ ]:
import pickle
# load up PPC results
with open(res_dir / "dsbmm_ppc_results.pkl", "rb") as f:
    dsbmm_ppc_results = pickle.load(f)
with open(res_dir / "dpf_ppc_results.pkl", "rb") as f:
    dpf_ppc_results = pickle.load(f)
with open(res_dir / "dpf_auc_results.pkl", "rb") as f:
    dpf_auc_results = pickle.load(f)